hello

In [81]:
import numpy as np
import statsmodels.stats as sms
from scipy import stats

#import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
#import statsmodels.api as sma
#from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, \
   #pearsonr, spearmanr, kendalltau, f_oneway, kruskal
#from statsmodels.stats.proportion import proportions_ztest


alpha = 0.05                     # допустимая вероятность ошибки I рода
beta = 0.2                      # допустимая вероятность ошибки II рода
mu_control = 1300               # средняя выручка с пользователя в контрольной группе
effect = 1300*0.05                    # ожидаемый размер эффекта
mu_pilot = mu_control + effect  # средняя выручка с пользователя в экспериментальной группе
std = 800                       # стандартное отклонение

# исторические данные выручки для 10000 клиентов
values = np.random.normal(mu_control, std, 10000)

def estimate_sample_size(effect, std, alpha, beta):
    """Оценка необходимого размер групп."""
    t_alpha = stats.norm.ppf(1 - alpha / 2, loc=0, scale=1)
    t_beta = stats.norm.ppf(1 - beta, loc=0, scale=1)
    var = 2 * std ** 2
    sample_size = int((t_alpha + t_beta) ** 2 * var / (effect ** 2))
    return sample_size

cohen_d = effect / std


estimated_std = np.std(values)
sample_size = estimate_sample_size(effect, estimated_std, alpha, beta)
sample_size1 = sms.power.TTestIndPower().solve_power(effect_size=cohen_d, alpha=alpha, power=1-beta, alternative='two-sided')
 

print(f'оценка необходимого размера групп = {sample_size}')
print(f'оценка необходимого размера групп 1 = {sample_size1}')

оценка необходимого размера групп = 2352
оценка необходимого размера групп 1 = 2378.840414004076


In [82]:
def run_synthetic_experiments(values, sample_size, effect=0, n_iter=10000):
    """Проводим синтетические эксперименты, возвращаем список p-value."""
    pvalues = []
    for _ in range(n_iter):
        a, b = np.random.choice(values, size=(2, sample_size,), replace=False)
        b += effect
        pvalue = stats.ttest_ind(a, b).pvalue
        pvalues.append(pvalue)
    return np.array(pvalues)

def print_estimated_errors(pvalues_aa, pvalues_ab, alpha):
    """Оценивает вероятности ошибок."""
    estimated_first_type_error = np.mean(pvalues_aa < alpha)
    estimated_second_type_error = np.mean(pvalues_ab >= alpha)
    ci_first = estimate_ci_bernoulli(estimated_first_type_error, len(pvalues_aa))
    ci_second = estimate_ci_bernoulli(estimated_second_type_error, len(pvalues_ab))
    print(f'оценка вероятности ошибки I рода = {estimated_first_type_error:0.4f}')
    print(f'  доверительный интервал = [{ci_first[0]:0.4f}, {ci_first[1]:0.4f}]')
    print(f'оценка вероятности ошибки II рода = {estimated_second_type_error:0.4f}')
    print(f'  доверительный интервал = [{ci_second[0]:0.4f}, {ci_second[1]:0.4f}]')

def estimate_ci_bernoulli(p, n, alpha=0.05):
    """Доверительный интервал для Бернуллиевской случайной величины."""
    t = stats.norm.ppf(1 - alpha / 2, loc=0, scale=1)
    std_n = np.sqrt(p * (1 - p) / n)
    return p - t * std_n, p + t * std_n

pvalues_aa = run_synthetic_experiments(values, sample_size, effect=0)
pvalues_ab = run_synthetic_experiments(values, sample_size, effect=effect)
print_estimated_errors(pvalues_aa, pvalues_ab, alpha)

оценка вероятности ошибки I рода = 0.0484
  доверительный интервал = [0.0442, 0.0526]
оценка вероятности ошибки II рода = 0.2037
  доверительный интервал = [0.1958, 0.2116]


In [39]:
import statsmodels.stats as sms
from scipy.stats import norm
# Параметры
alpha = 0.05  # уровень значимостиal
alpha = 0.2   # мощность теста
effect_size = 0.05  # МДЕ в процентах
baseline_mean = values.mean()  # средняя выручка
std_dev = values.std()  # стандартное отклонение

print('std =', std_dev)
z_alpha = norm.ppf(1 - alpha / 2)
z_beta = norm.ppf(power)  
print(z_alpha)
print(z_beta)
# Расчет минимального эффекта (в абсолютных единицах)
md_effect = baseline_mean * effect_size
print(md_effect)
# Расчет эффекта в терминах стандартного отклонения (Cohen's d)
cohen_d = md_effect / std_dev
print(cohen_d)
# Расчет размера выборки для двухвыборочного t-теста
sample_size = sms.power.TTestIndPower().solve_power(effect_size=cohen_d, alpha=alpha, power=1-alpha, alternative='two-sided')

# Вывод размера выборки
print(f"Необходимый размер выборки для каждой группы: {round(sample_size)}")

std = 785.2803371318107


NameError: name 'power' is not defined